In [1]:
import os
import sys
import aim

os.environ["CONFIG_PATHS"] = "../configs/self_play.yaml"
os.environ["CONFIG_OVERRIDES"] = 'game.moves_directory="../data/moves_10"'
sys.path.append("../src")

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import time
from torch.utils.data import TensorDataset, DataLoader
from display import Display
from configuration import moves_data, config
from training.actor import TrainingActor
import training.helpers

/Users/shivamsarodia/.pyenv/versions/3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-26 22:46:14,722	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Loaded config:  {"development": {"debug_mode": true, "profile": false, "runtime": 72000, "display_logs_in_console": false, "output_directory": "data/2024-11-24_00-18-45-literatist"}, "logging": {"save_interval": 3600, "mcts_report_fraction": 0, "ucb_report": false, "gpu_evaluation": true, "made_move": true}, "game": {"board_size": 10, "num_moves": 6233, "moves_directory": "../data/moves_10"}, "architecture": {"gameplay_processes": 6, "coroutines_per_process": 256, "game_flush_threshold": 200}, "training": {"run": true, "network_name": "default", "batch_size": 64, "policy_loss_weight": 0.158, "learning_rate": 0.001, "sample_window": 50000, "samples_per_generation": 10000, "sampling_ratio": 1.0, "minimum_window_size": 10000, "new_data_check_interval": 30}, "networks": {"default": {"main_body_channels": 64, "value_head_channels": 16, "value_head_flat_layer_width": 64, "policy_head_channels": 64, "residual_blocks": 8, "model_path": "", "model_directory": "data/2024-11-24_00-18-45-literatis

In [2]:
from neural_net import NeuralNet
from training.game_data_manager import GameDataManager

In [3]:
NETWORK_CONFIG = config()["networks"]["default"]

In [4]:
BATCH_SIZE = 64
DEVICE = "mps"
MOVES = moves_data()
GAMES_DIR = "../data/2024-11-24_00-18-45-literatist/games"
WINDOW_SIZE = 50000
MINIMUM_WINDOW_SIZE = 10000
POLICY_LOSS_WEIGHT = 0.158
LEARNING_RATE = 1e-3
SAMPLING_RATIO = 1.0

Loading file: piece_indices
Loading file: rotation_mapping
Loading file: new_occupieds
Loading file: moves_ruled_out_for_all
Loading file: scores
Loading file: moves_ruled_out_for_player
Loading file: moves_enabled_for_player
Loading file: new_adjacents
Loading file: new_corners


In [5]:
game_data_manager = GameDataManager(GAMES_DIR, 50000)

In [6]:
model = NeuralNet(NETWORK_CONFIG)
model.to(DEVICE)
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [7]:
training.helpers.feed_window_until_amount(
    game_data_manager,
    642275,
    1e6,
)
print("Window size: ", game_data_manager.current_window_size())
print("Cumulative window fed: ", game_data_manager.cumulative_window_fed())

Window size:  50000
Cumulative window fed:  642275


In [13]:
run = aim.Run(repo='/tmp/.aim')

batch_index = 0
while True:
    if batch_index % 100 == 0:
        print(f"Batch {batch_index}, window size {game_data_manager.current_window_size()}, cumulative window fed {game_data_manager.cumulative_window_fed()}")

    training_result = training.helpers.loop_iteration(
        model,
        optimizer,
        game_data_manager,
        device=DEVICE,
        batch_size=BATCH_SIZE,
        sampling_ratio=SAMPLING_RATIO,
        policy_loss_weight=POLICY_LOSS_WEIGHT,
    )
    if not training_result:
        break

    for key, value in training_result.items():
        run.track(
            value,
            name=key,
            step=batch_index,
        )
    batch_index += 1

run.close()

Batch 0, window size 10000, cumulative window fed 10000
Batch 100, window size 16400, cumulative window fed 16400
Batch 200, window size 22800, cumulative window fed 22800
Batch 300, window size 29200, cumulative window fed 29200
Batch 400, window size 35600, cumulative window fed 35600
Batch 500, window size 42000, cumulative window fed 42000
Batch 600, window size 48400, cumulative window fed 48400
Batch 700, window size 50000, cumulative window fed 54800
Batch 800, window size 50000, cumulative window fed 61200
Batch 900, window size 50000, cumulative window fed 67600
Batch 1000, window size 50000, cumulative window fed 74000
Batch 1100, window size 50000, cumulative window fed 80400
Batch 1200, window size 50000, cumulative window fed 86800
Batch 1300, window size 50000, cumulative window fed 93200
Batch 1400, window size 50000, cumulative window fed 99600
Batch 1500, window size 50000, cumulative window fed 106000
Batch 1600, window size 50000, cumulative window fed 112400
Batch 1

ValueError: need at least one array to concatenate

In [14]:
torch.save(model.state_dict(), "../data/2024-11-26_19-59-19-prodigal/models/trained_in_notebook.pt")

In [8]:
# game_paths = [
#     os.path.join(GAMES_DIR, f)
#     for f in os.listdir(GAMES_DIR)
# ]

# # Train on first 3/4, test on last 1/4.
# train_path_index_cutoff = 3 * len(game_paths) // 4

# train_paths = sorted(game_paths)[:train_path_index_cutoff]
# test_paths = sorted(game_paths)[train_path_index_cutoff:]

# def paths_to_dataloader(paths):
#     boards, policies, values = load_old_format_games(paths)
#     dataset = TensorDataset(
#         torch.Tensor(boards),
#         torch.Tensor(policies),
#         torch.Tensor(values),
#     )
#     return DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# train_dataloader = paths_to_dataloader(train_paths)
# test_dataloader = paths_to_dataloader(test_paths)

In [9]:
# run = aim.Run(repo='/tmp/.aim')

# model = NeuralNet(config()["networks"]["default"]).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# for batch_index, (boards, policies, values) in tqdm(enumerate(train_dataloader)):
#     boards = boards.to(DEVICE)
#     policies = policies.to(DEVICE)
#     values = values.to(DEVICE)

#     pred_values, pred_policy = model(boards)
#     value_loss = nn.CrossEntropyLoss()(
#         pred_values,
#         values,
#     )
#     policy_loss = 0.158 * nn.CrossEntropyLoss()(
#         pred_policy,
#         policies,
#     )
#     loss = value_loss + policy_loss

#     run.track(
#         value_loss.item(),
#         name="value_loss",
#         step=batch_index,
#     )
#     run.track(
#         policy_loss.item(),
#         name="policy_loss",
#         step=batch_index,
#     )
#     run.track(
#         loss.item(),
#         name="total_loss",
#         step=batch_index,
#     )

#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()

# run.close()

In [10]:
# value_losses = []
# policy_losses = []
# total_losses = []

# for batch_index, (boards, policies, values) in tqdm(enumerate(test_dataloader)):
#     pred_values, pred_policy = model(boards)
#     value_loss = nn.CrossEntropyLoss()(
#         pred_values,
#         values,
#     )
#     policy_loss = 0.158 * nn.CrossEntropyLoss()(
#         pred_policy,
#         policies,
#     )
#     loss = value_loss + policy_loss

#     value_losses.append(value_loss.item())
#     policy_losses.append(policy_loss.item())
#     total_losses.append(loss.item())

# print("Average value loss: ", sum(value_losses) / len(value_losses))
# print("Average policy loss: ", sum(policy_losses) / len(policy_losses))
# print("Average total loss: ", sum(total_losses) / len(total_losses))